In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('C:\\Users\\ywang\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\EIR runs\\Alt1 (s26) runs\\Alt1_v3_test_far_tiers_FINAL_EIR_ALT')

### calculate non-residential-sqft and job-space growth from buildings output

In [3]:
# simplify building types and sources

blg_type_dict = {'HO':  'other',
                 'OF':  'office',
                 'PA2': 'other',
                 'RS':  'retail',
                 'HM':  'other',
                 'HS':  'other',
                 'IH':  'industrial/warehouse',
                 'IL':  'industrial/warehouse',
                 'IW':  'industrial/warehouse',
                 'MR':  'mixed-use',
                 'SC':  'other',
                 'RB':  'retail',
                 'HT':  'other',
                 'ME':  'mixed-use',
                 'GQ':  'other',
                 'VP':  'other',
                 'PG':  'other',
                 'VA':  'other',
                 'OT':  'other'}

In [4]:
pg = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Current PBA50 Large General Input Data\2021_02_25_parcels_geography.csv',
                 usecols = ['PARCEL_ID', 'eir_tra_id', 'juris'])
juris = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\petrale\zones\jurisdictions\juris_county_id.csv',
                    usecols = ['juris_name_full', 'county_name'])


# calculate non-residential-sqft and job-spaces growth
b50 = pd.read_csv('run375_building_data_2050.csv',
                  usecols = ['building_id', 'parcel_id', 'non_residential_sqft', 'job_spaces', 'building_type', 'source'])
b50['blg_type'] = b50['building_type'].map(blg_type_dict)
b50 = b50.merge(pg, left_on='parcel_id', right_on='PARCEL_ID', how='left')
b50 = b50.merge(juris, left_on = 'juris', right_on = 'juris_name_full', how='left')

# check NA in columns
for i in list(b50):
    if b50[i].isnull().values.any():
        print('column with NA: {}'.format(i))

# fill na in tra_id
b50['eir_tra_id'] = b50['eir_tra_id'].fillna('NA')

b50_sum = b50.groupby(['county_name', 'juris', 'eir_tra_id', 'blg_type', 'source'])['non_residential_sqft', 'job_spaces'].sum().reset_index()
b50_sum.rename(columns = {'non_residential_sqft': 'non_residential_sqft_50',
                          'job_spaces': 'job_spaces_50'}, inplace=True)


print(b50[['non_residential_sqft','job_spaces']].sum())
print(b50_sum[['non_residential_sqft_50','job_spaces_50']].sum())
# display(b50_sum)


b15 = pd.read_csv('run375_building_data_2015.csv',
                  usecols = ['building_id', 'parcel_id', 'non_residential_sqft', 'job_spaces', 'building_type', 'source'])
b15['blg_type'] = b15['building_type'].map(blg_type_dict)
b15 = b15.merge(pg, left_on='parcel_id', right_on='PARCEL_ID', how='left')
b15 = b15.merge(juris, left_on = 'juris', right_on = 'juris_name_full', how='left')

# check NA in columns
for i in list(b15):
    if b15[i].isnull().values.any():
        print('column with NA: {}'.format(i))
    
# fill na in tra_id
b15['eir_tra_id'] = b15['eir_tra_id'].fillna('NA')

b15_sum = b15.groupby(['county_name', 'juris', 'eir_tra_id', 'blg_type', 'source'])['non_residential_sqft', 'job_spaces'].sum().reset_index()
b15_sum.rename(columns = {'non_residential_sqft': 'non_residential_sqft_15',
                          'job_spaces': 'job_spaces_15'}, inplace=True)

print(b15[['non_residential_sqft','job_spaces']].sum())
print(b15_sum[['non_residential_sqft_15','job_spaces_15']].sum())
# display(b15_sum)

column with NA: PARCEL_ID
column with NA: juris
column with NA: eir_tra_id
column with NA: juris_name_full
column with NA: county_name


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


non_residential_sqft    2.443263e+09
job_spaces              5.706429e+06
dtype: float64
non_residential_sqft_50    2.442822e+09
job_spaces_50              5.705115e+06
dtype: float64
column with NA: PARCEL_ID
column with NA: juris
column with NA: eir_tra_id
column with NA: juris_name_full
column with NA: county_name


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


non_residential_sqft    2.281374e+09
job_spaces              4.971945e+06
dtype: float64
non_residential_sqft_15    2.280933e+09
job_spaces_15              4.970658e+06
dtype: float64


In [5]:
b_chg = b50_sum.merge(b15_sum, on=['county_name','juris','eir_tra_id','blg_type','source'], how='outer')

for i in ['non_residential_sqft_50', 'non_residential_sqft_15', 'job_spaces_50', 'job_spaces_15']:
    b_chg[i] = b_chg[i].fillna(0)
    
b_chg['non_residential_sqft_growth'] = b_chg['non_residential_sqft_50'] - b_chg['non_residential_sqft_15']
b_chg['job_spaces_growth'] = b_chg['job_spaces_50'] - b_chg['job_spaces_15']

b_chg.to_csv('growth_by_building_Alt1.csv', index=False)

In [6]:
b_chg_type = b_chg[['blg_type',
                    'source',
                    'non_residential_sqft_growth',
                    'job_spaces_growth']].groupby(['blg_type',
                                                   'source'])['non_residential_sqft_growth',
                                                              'job_spaces_growth'].sum().reset_index().sort_values(['blg_type', 'source'])
b_chg_type['non_residential_sqft_growth'] = b_chg_type['non_residential_sqft_growth'].apply(lambda x: int(x))
display(b_chg_type)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,blg_type,source,non_residential_sqft_growth,job_spaces_growth
0,industrial/warehouse,bas_bp_new,2708101,3739.0
1,industrial/warehouse,basis,5634382,8099.0
2,industrial/warehouse,cs,43546900,52931.0
3,industrial/warehouse,h5_inputs,-66223205,-65614.0
4,industrial/warehouse,incubator,11250000,11868.0
5,industrial/warehouse,manual,5920000,9744.0
6,industrial/warehouse,ppa,45701976,48392.0
7,mixed-use,bas_bp_new,342433,644.0
8,mixed-use,basis,16229376,47284.0
9,mixed-use,h5_inputs,-36909358,-55355.0


In [7]:
b_chg.loc[b_chg.eir_tra_id.isnull(), 'eir_tra_id'] = 'non-tra'
b_chg_geo = b_chg[['juris',
                   'eir_tra_id',
                   'non_residential_sqft_growth',
                   'job_spaces_growth']].groupby(['eir_tra_id',
                                                  'juris'])['non_residential_sqft_growth',
                                                            'job_spaces_growth'].sum().reset_index().sort_values(['eir_tra_id', 'juris'])
b_chg_geo['non_residential_sqft_growth'] = b_chg_geo['non_residential_sqft_growth'].apply(lambda x: int(x))
display(b_chg_geo)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


,eir_tra_id,juris,non_residential_sqft_growth,job_spaces_growth
0,NA,alameda,307520,1446.0
1,NA,albany,0,0.0
2,NA,american_canyon,4514275,5329.0
3,NA,antioch,-241475,-463.0
4,NA,atherton,0,90.0
...,...,...,...,...
261,tra3,union_city,352408,1330.0
262,tra3,vacaville,1743482,4626.0
263,tra3,vallejo,3168566,3449.0
264,tra3,walnut_creek,-2506899,-6098.0


In [8]:
b_chg_geo_type = b_chg[['juris',
                        'blg_type',
                        'source',
                        'non_residential_sqft_growth',
                        'job_spaces_growth']].groupby(['juris',
                                                       'blg_type',
                                                       'source'])['non_residential_sqft_growth',
                                                                  'job_spaces_growth'].sum().reset_index().sort_values(['blg_type', 'source'])
b_chg_geo_type['non_residential_sqft_growth'] = b_chg_geo_type['non_residential_sqft_growth'].apply(lambda x: int(x))
display(b_chg_geo_type)

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,juris,blg_type,source,non_residential_sqft_growth,job_spaces_growth
30,american_canyon,industrial/warehouse,bas_bp_new,394540,628.0
118,brentwood,industrial/warehouse,bas_bp_new,33664,33.0
173,campbell,industrial/warehouse,bas_bp_new,0,0.0
221,concord,industrial/warehouse,bas_bp_new,52523,85.0
429,fremont,industrial/warehouse,bas_bp_new,51434,102.0
...,...,...,...,...,...
1099,san_carlos,retail,manual,73020,178.0
1129,san_francisco,retail,manual,3117000,7728.0
1161,san_jose,retail,manual,500000,1296.0
1203,san_mateo,retail,manual,42191,105.0


### calculate job-space and emp growth from parcels output

In [9]:
p50 = pd.read_csv('run375_parcel_data_2050.csv',
                  usecols = ['parcel_id', 'total_job_spaces', 'totemp'])
p50 = p50.merge(pg, left_on='parcel_id', right_on='PARCEL_ID', how='left')
p50 = p50.merge(juris, left_on = 'juris', right_on = 'juris_name_full', how='left')

# check NA in columns
for i in list(p50):
    if p50[i].isnull().values.any():
        print('column with NA: {}'.format(i))
    
# fill na in tra_id
p50['eir_tra_id'] = p50['eir_tra_id'].fillna('NA')
p50['totemp'] = p50['totemp'].fillna(0)

p50_sum = p50.groupby(['county_name', 'juris', 'eir_tra_id'])['total_job_spaces', 'totemp'].sum().reset_index()
p50_sum.rename(columns = {'total_job_spaces': 'total_job_spaces_50',
                          'totemp': 'totemp_50'}, inplace=True)

print(p50[['total_job_spaces', 'totemp']].sum())
print(p50_sum[['total_job_spaces_50', 'totemp_50']].sum())



p15 = pd.read_csv('run375_parcel_data_2015.csv',
                  usecols = ['parcel_id', 'total_job_spaces', 'totemp'])
p15 = p15.merge(pg, left_on='parcel_id', right_on='PARCEL_ID', how='left')
p15 = p15.merge(juris, left_on = 'juris', right_on = 'juris_name_full', how='left')

# check NA in columns
for i in list(p15):
    if p15[i].isnull().values.any():
        print('column with NA: {}'.format(i))

# fill na in tra_id
p15['eir_tra_id'] = p15['eir_tra_id'].fillna('NA')
p15['totemp'] = p15['totemp'].fillna(0)

p15_sum = p15.groupby(['county_name', 'juris', 'eir_tra_id'])['total_job_spaces', 'totemp'].sum().reset_index()
p15_sum.rename(columns = {'total_job_spaces': 'total_job_spaces_15',
                          'totemp': 'totemp_15'}, inplace=True)

print(p15[['total_job_spaces', 'totemp']].sum())
print(p15_sum[['total_job_spaces_15', 'totemp_15']].sum())

column with NA: totemp
column with NA: PARCEL_ID
column with NA: juris
column with NA: eir_tra_id
column with NA: juris_name_full
column with NA: county_name


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  from ipykernel import kernelapp as app


total_job_spaces    5706429.0
totemp              5408460.0
dtype: float64
total_job_spaces_50    5705115.0
totemp_50              5407146.0
dtype: float64
column with NA: totemp
column with NA: PARCEL_ID
column with NA: juris
column with NA: eir_tra_id
column with NA: juris_name_full
column with NA: county_name


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


total_job_spaces    4971945.0
totemp              4005318.0
dtype: float64
total_job_spaces_15    4970658.0
totemp_15              4004175.0
dtype: float64


In [10]:
p_chg = p15_sum.merge(p50_sum, on=['county_name', 'juris', 'eir_tra_id'], how='outer')

for i in ['total_job_spaces_50', 'total_job_spaces_15', 'totemp_50', 'totemp_15']:
    p_chg[i] = p_chg[i].fillna(0)
    
p_chg['total_job_spaces_growth'] = p_chg['total_job_spaces_50'] - p_chg['total_job_spaces_15']
p_chg['totemp_growth'] = p_chg['totemp_50'] - p_chg['totemp_15']

p_chg.to_csv('growth_by_parcel_Alt1.csv', index=False)